In [170]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# #5 Ranking and Hybrid Recommender Systems

[Построение длинного и короткого портфеля акций с помощью
нового алгоритма listwise learn-to-ranking](https://arxiv.org/pdf/2104.12484.pdf)_(pdf)_

[Recommender system using Bayesian personalized ranking](https://towardsdatascience.com/recommender-system-using-bayesian-personalized-ranking-d30e98bba0b9)_(towardsdatascience)_

[Intro to WARP Loss, automatic differentiation and PyTorch](https://medium.com/@gabrieltseng/intro-to-warp-loss-automatic-differentiation-and-pytorch-b6aa5083187a)_(towardsdatascience)_

In [8]:
!exit;conda -h

In [171]:
import os, sys
import pandas as pd
import numpy as np
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

from lightfm import LightFM
from lightfm.evaluation import precision_at_k


module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.utils import prefilter_items
print('Done')

Done


In [172]:
data = pd.read_csv('../data/retail_train.csv')

item_features = pd.read_csv('../features_data/product.csv')
user_features = pd.read_csv('../features_data/hh_demographic.csv')

In [173]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [174]:
# prefilter data
print('{:=^79}'.format('Train Sample'))
data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)


# prepare CSR train data
user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='count', 
                                  fill_value=0
                                 ).astype(float) 
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

# prepare CSR test data
print('{:=^79}'.format('Test Sample'))
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

data_test_filtered = prefilter_items(data_test, take_n_popular=5000, item_features=item_features)

test_user_item_matrix = pd.pivot_table(data_test_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )
test_user_item_matrix = test_user_item_matrix.astype(float)

=================================Train Sample==================================
== Starting prefilter info ==
shape: (2278490, 12)
# users: 2499
# items: 86865
Sparsity: 1.050%
== Ending prefilter info ==
shape: (541583, 13)
# users: 2455
# items: 5000
Sparsity: 4.412%
new_columns: {'price'}
==================================Test Sample==================================
== Starting prefilter info ==
shape: (115644, 12)
# users: 2041
# items: 22143
Sparsity: 0.256%
== Ending prefilter info ==
shape: (33942, 13)
# users: 1847
# items: 5000
Sparsity: 0.368%
new_columns: {'price'}


In [175]:
# prepare dicts
userids = np.union1d(user_item_matrix.index.values, test_user_item_matrix.index.values)
itemids = np.union1d(user_item_matrix.columns.values, test_user_item_matrix.columns.values)

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [176]:
# prepare user/items features
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)

item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)


# encoding features
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

## *Grid Search with `hyperopt`*

In [182]:
# %pip install hyperopt

In [178]:
def evaluate_model(model, filtered_data, user_features, item_features) -> float:
    user_item_matrix = pd.pivot_table(
        filtered_data, 
        index='user_id', columns='item_id', 
        values='price',
        aggfunc='sum', 
        fill_value=0
    ).astype(float)
    sparse_user_item = csr_matrix(user_item_matrix).tocsr()

    user_feat = pd.DataFrame(user_item_matrix.index)
    user_feat = user_feat.merge(user_features, on='user_id', how='left')
    user_feat.set_index('user_id', inplace=True)
    item_feat = pd.DataFrame(user_item_matrix.columns)
    item_feat = item_feat.merge(item_features, on='item_id', how='left')
    item_feat.set_index('item_id', inplace=True)

    user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
    item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

    model.fit(
          (sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4,
          verbose=False
          )

    p = precision_at_k(
        model, 
        sparse_user_item, 
        user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
        item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
        k=5).mean()

    return p, model

In [179]:
from hyperopt import hp, Trials, fmin, tpe, STATUS_OK
from math import e

def hyperopt_obj(params, data):
    t = params['type']
    del params['type']
    if t == 'lightfm':
        clf = LightFM(**params)
    else:
        return 0
    
    return evaluate_model(model=clf, 
                          filtered_data=data,
                          user_features=user_features,
                          item_features=item_features)

In [180]:
space = hp.choice('task', 
    [
        {'type': 'lightfm',
         'loss': hp.choice('loss', ['warp', 'bpr']),
         'no_components': hp.choice('n_components', range(40, 150)),
         'learning_rate': hp.choice('learning_rate', [.005, .01, .1, .15,]),
         'max_sampled': hp.choice('max_sampled', range(5, 16)),
         'epsilon': hp.uniform('epsilon', 1e-6, 1e-2),
         'item_alpha': hp.uniform('item_alpha', 1e-3, 0.1),
         'user_alpha': hp.uniform('user_alpha', 1e-3, 0.1),
         'random_state': hp.choice('random_state', [111])}
    ])

count = 0
best = 0
def f(params):
    global best, count
    best_acc_delta = 10e3 # I want to get minimal train and test accuracy delta 
    count += 1
    acc_train, model = hyperopt_obj(params.copy(), data_train_filtered)
    acc_test = precision_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                k=5).mean()
    acc_delta = abs(acc_train - acc_test)
    if acc_delta < best_acc_delta:
        print('{:=^79}'.format(f'new best! train: {acc_train} test: {acc_test}'))
        print('using: {}'.format(model.get_params()))
        best_acc_delta = acc_delta
        best = acc_test

    return {'loss': -acc_test, 'status': STATUS_OK}


trials = Trials()
best = fmin(f, space, algo=tpe.suggest, max_evals=200, trials=trials)
print(f'best:\n{best}')

========new best! train: 0.21507127583026886 test: 0.001299404539167881========                                                                                                  
using: {'loss': 'warp', 'learning_schedule': 'adagrad', 'no_components': 43, 'learning_rate': 0.01, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.00041026111888976363, 'max_sampled': 11, 'item_alpha': 0.0743631881508599, 'user_alpha': 0.09244962299898474, 'random_state': RandomState(MT19937) at 0x7FBB60216A40}
=======new best! train: 0.24529534578323364 test: 0.0011911207111552358========                                                                                                  
using: {'loss': 'warp', 'learning_schedule': 'adagrad', 'no_components': 85, 'learning_rate': 0.01, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.005589805173075095, 'max_sampled': 5, 'item_alpha': 0.060241609666262956, 'user_alpha': 0.06631710116952333, 'random_state': RandomState(MT19937) at 0x7FBB60216A40}
========new best! train: 0.

========new best! train: 0.224358469247818 test: 0.0015159719623625278=========                                                                                                  
using: {'loss': 'warp', 'learning_schedule': 'adagrad', 'no_components': 102, 'learning_rate': 0.01, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.004210832803694575, 'max_sampled': 9, 'item_alpha': 0.05199185739299922, 'user_alpha': 0.09286136019606236, 'random_state': RandomState(MT19937) at 0x7FBB60216A40}
=======new best! train: 0.22052954137325287 test: 0.0011911207111552358========                                                                                                  
using: {'loss': 'bpr', 'learning_schedule': 'adagrad', 'no_components': 72, 'learning_rate': 0.01, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.007246592575687156, 'max_sampled': 13, 'item_alpha': 0.05954893806132233, 'user_alpha': 0.0271540555484134, 'random_state': RandomState(MT19937) at 0x7FBB60216A40}
========new best! train: 0.252

=======new best! train: 0.17384928464889526 test: 0.0009745533461682498========                                                                                                  
using: {'loss': 'bpr', 'learning_schedule': 'adagrad', 'no_components': 91, 'learning_rate': 0.01, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.007791823407584761, 'max_sampled': 11, 'item_alpha': 0.0555057009626669, 'user_alpha': 0.07767757531956967, 'random_state': RandomState(MT19937) at 0x7FBB39902A40}
========new best! train: 0.2502647638320923 test: 0.0014076882507652044========                                                                                                  
using: {'loss': 'bpr', 'learning_schedule': 'adagrad', 'no_components': 69, 'learning_rate': 0.15, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.00942146263628851, 'max_sampled': 12, 'item_alpha': 0.04529323898527663, 'user_alpha': 0.010861254795860559, 'random_state': RandomState(MT19937) at 0x7FBAECEA5240}
========new best! train: 0.2338

========new best! train: 0.16179226338863373 test: 0.003789929673075676========                                                                                                  
using: {'loss': 'warp', 'learning_schedule': 'adagrad', 'no_components': 78, 'learning_rate': 0.15, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.007941659877243, 'max_sampled': 14, 'item_alpha': 0.09965521429037298, 'user_alpha': 0.04022932256656131, 'random_state': RandomState(MT19937) at 0x7FBAECEA5240}
=======new best! train: 0.24187374114990234 test: 0.0036816459614783525========                                                                                                  
using: {'loss': 'bpr', 'learning_schedule': 'adagrad', 'no_components': 96, 'learning_rate': 0.005, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.002131930493656251, 'max_sampled': 9, 'item_alpha': 0.01937315765656205, 'user_alpha': 0.005091779651387393, 'random_state': RandomState(MT19937) at 0x7FBB39902A40}
========new best! train: 0.0112

========new best! train: 0.25792261958122253 test: 0.004764482844620943========                                                                                                  
using: {'loss': 'bpr', 'learning_schedule': 'adagrad', 'no_components': 127, 'learning_rate': 0.15, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.00870528875602789, 'max_sampled': 8, 'item_alpha': 0.03463897359733837, 'user_alpha': 0.01369235273507129, 'random_state': RandomState(MT19937) at 0x7FBB39902A40}
=======new best! train: 0.16358453035354614 test: 0.0017325392691418529========                                                                                                  
using: {'loss': 'bpr', 'learning_schedule': 'adagrad', 'no_components': 136, 'learning_rate': 0.15, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.009537915275057974, 'max_sampled': 8, 'item_alpha': 0.041530160168280796, 'user_alpha': 0.02091749984368083, 'random_state': RandomState(MT19937) at 0x7FBAECEA5240}
========new best! train: 0.163

...

========new best! train: 0.22761709988117218 test: 0.001299404539167881========                                                                                                  
using: {'loss': 'warp', 'learning_schedule': 'adagrad', 'no_components': 104, 'learning_rate': 0.005, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.008300950876167435, 'max_sampled': 6, 'item_alpha': 0.03965042263506874, 'user_alpha': 0.09310201337840004, 'random_state': RandomState(MT19937) at 0x7FBAECEA5240}
========new best! train: 0.04594704881310463 test: 0.002815376268699765========                                                                                                  
using: {'loss': 'bpr', 'learning_schedule': 'adagrad', 'no_components': 97, 'learning_rate': 0.15, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.007839415546733607, 'max_sampled': 8, 'item_alpha': 0.03628687336444096, 'user_alpha': 0.0801845004374418, 'random_state': RandomState(MT19937) at 0x7FBB39902A40}
=======new best! train: 0.2418

========new best! train: 0.18183299899101257 test: 0.004656199365854263========                                                                                                  
using: {'loss': 'bpr', 'learning_schedule': 'adagrad', 'no_components': 119, 'learning_rate': 0.1, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.009683838068381984, 'max_sampled': 8, 'item_alpha': 0.05604364768489546, 'user_alpha': 0.0331127379570733, 'random_state': RandomState(MT19937) at 0x7FBB39902A40}
=======new best! train: 0.012871691025793552 test: 0.0017325392691418529=======                                                                                                  
using: {'loss': 'bpr', 'learning_schedule': 'adagrad', 'no_components': 108, 'learning_rate': 0.15, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.005843557043097391, 'max_sampled': 7, 'item_alpha': 0.02390452091549968, 'user_alpha': 0.049229992765525185, 'random_state': RandomState(MT19937) at 0x7FBAECEA5240}
========new best! train: 0.1714

=======new best! train: 0.012545824982225895 test: 0.0011911207111552358=======                                                                                                  
using: {'loss': 'bpr', 'learning_schedule': 'adagrad', 'no_components': 123, 'learning_rate': 0.15, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.007705739083671214, 'max_sampled': 6, 'item_alpha': 0.07428168206953335, 'user_alpha': 0.02359037282563027, 'random_state': RandomState(MT19937) at 0x7FBAECEA5240}
=======new best! train: 0.007983705960214138 test: 0.0021656739991158247=======                                                                                                  
using: {'loss': 'bpr', 'learning_schedule': 'adagrad', 'no_components': 121, 'learning_rate': 0.15, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.0074667667456326785, 'max_sampled': 8, 'item_alpha': 0.0634523838646242, 'user_alpha': 0.029815366794253815, 'random_state': RandomState(MT19937) at 0x7FBB39902A40}
=======new best! train: 0.007

========new best! train: 0.01319755706936121 test: 0.003140227170661092========                                                                                                  
using: {'loss': 'bpr', 'learning_schedule': 'adagrad', 'no_components': 145, 'learning_rate': 0.15, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.006689761657460857, 'max_sampled': 13, 'item_alpha': 0.066559801536724, 'user_alpha': 0.027422214971872708, 'random_state': RandomState(MT19937) at 0x7FBB39902A40}
=======new best! train: 0.21580450236797333 test: 0.0014076882507652044========                                                                                                  
using: {'loss': 'bpr', 'learning_schedule': 'adagrad', 'no_components': 51, 'learning_rate': 0.1, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.008391447565528348, 'max_sampled': 8, 'item_alpha': 0.027500460916155163, 'user_alpha': 0.00686395738653188, 'random_state': RandomState(MT19937) at 0x7FBAECEA5240}
========new best! train: 0.2981

========new best! train: 0.1684725135564804 test: 0.002273957710713148=========                                                                                                  
using: {'loss': 'bpr', 'learning_schedule': 'adagrad', 'no_components': 121, 'learning_rate': 0.005, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.0005989418435868101, 'max_sampled': 13, 'item_alpha': 0.008808903723818115, 'user_alpha': 0.024495337869686426, 'random_state': RandomState(MT19937) at 0x7FBAECEA5240}
=======new best! train: 0.24187374114990234 test: 0.0036816459614783525========                                                                                                  
using: {'loss': 'bpr', 'learning_schedule': 'adagrad', 'no_components': 107, 'learning_rate': 0.005, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 0.0019017248822003133, 'max_sampled': 13, 'item_alpha': 0.029337665452551325, 'user_alpha': 0.007936568870948886, 'random_state': RandomState(MT19937) at 0x7FBB39902A40}
=======new best! tra

In [183]:
best = {'loss': 'bpr', 
        'no_components': 75, 
        'learning_rate': 0.1,
        'epsilon': 0.00037744557464881477, 
        'max_sampled': 5, 
        'item_alpha': 0.010879605896560014, 
        'user_alpha': 0.03401434201826126, 
        'random_state': 0}

## *Init model*

In [184]:
model = LightFM(**best)

## *Fit Train*

In [185]:
%%time

model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4,
          verbose=False) 

CPU times: user 3min 54s, sys: 408 ms, total: 3min 55s
Wall time: 1min


## *Evaluation*

In [186]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())

print('user_emb shapes:', user_emb[0].shape, user_emb[1].shape)
print('item_emb shapes:', item_emb[0].shape, item_emb[1].shape)

user_emb shapes: (2455,) (2455, 75)
item_emb shapes: (5000,) (5000, 75)


In [187]:
test_item_ids = np.array([1, 2, 3, 200, 1200, 4, 5])

predictions = model.predict(user_ids=0, item_ids=test_item_ids,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=4)
predictions

array([-231.96638, -232.30417, -232.56096, -232.52869, -232.30084,
       -232.48209, -232.49571], dtype=float32)

In [188]:
print('{} new items'.format(len([i for i in test_user_item_matrix.columns if i not in user_item_matrix.columns])))
print('{} new users'.format(len([i for i in test_user_item_matrix.index if i not in user_item_matrix.index])))

1589 new items
18 new users


In [189]:
users_purchases = data_train_filtered.copy()
users_purchases = users_purchases.groupby('user_id').apply(lambda df: list(map(lambda x: int(x), df.item_id.unique()))).reset_index(name='purchases')
users_purchases['purchases_count'] = users_purchases['purchases'].apply(lambda x: len(x))
users_purchases

,user_id,purchases,purchases_count
0,1,"[912676, 945805, 958046, 977545, 1043064, 1055...",142
1,2,"[1077555, 846833, 1133018, 824005, 941183, 845...",132
2,3,"[878996, 882830, 904360, 951590, 964968, 10920...",182
3,4,"[877523, 883932, 901032, 904973, 938566, 96496...",38
4,5,"[938983, 866292, 925054, 946308, 1022011, 1071...",35
...,...,...,...
2450,2496,"[886703, 956609, 1105467, 1138132, 866227, 955...",242
2451,2497,"[1101956, 900802, 951590, 1065067, 1119089, 88...",235
2452,2498,"[1031697, 1049832, 908531, 985480, 1025457, 82...",140
2453,2499,"[853197, 903230, 821344, 822346, 846830, 85167...",214


In [190]:
users_purchases.query('user_id == {}'.format(1)).purchases

0    [912676, 945805, 958046, 977545, 1043064, 1055...
Name: purchases, dtype: object

In [191]:
user_features.query('user_id == 2455')

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
788,45-54,B,25-34K,Homeowner,2 Adults No Kids,2,None/Unknown,2455


In [192]:
# from lightfm.py
user_ids = np.repeat(np.int32(0), len(test_item_ids))
user_ids.max()+1, user_features.shape[0]

(1, 801)

In [193]:
[i for i in users_purchases.user_id if i not in user_feat_lightfm.index]

[]

In [194]:
test_item_ids = [i for i in range(100)] 
    # как и где лучше взять тестовые значения?

for user_id in users_purchases.user_id:
    print('USER: {}'.format(user_id))
    bought = users_purchases.query('user_id == {}'.format(user_id)).purchases.values.tolist()[0]
    print('bought: {}'.format(len(bought)))
    predictions = model.predict(user_ids=user_id, item_ids=test_item_ids,
                                user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                num_threads=4)
    print(predictions.argsort(axis=-1))
    
    
#     когда юзер = 2455, алгоритм ругается на отсутствие его в
#     user-features датасете, хотя он есть. 

#     Буду благодарна за комментарии по этому поводу

USER: 1
bought: 142
[62 15 50 97  9 86 85 78 42 55 81 41 59 33 68 27 82 21 47 11 80 10 18 43
 45 29 38 35 90 89 63 73 88 60 16 54 36 44 31 72 53 58 92 26 40  5 37 93
 64 94 32 67 71 30 46  3  8 95 66 99 98 76 74 24 96 49 83 51 14  4  7 52
 17 39 13 48 61 23 65 91 87 79  2 22 57 19 20 75 25 34  6 56 77 12 28 84
 69  0  1 70]
USER: 2
bought: 132
[62 15 50 97  9 86 85 78 42 55 81 41 59 33 68 27 82 21 47 11 80 10 18 43
 45 29 38 35 90 89 63 73 88 60 16 54 36 44 31 72 53 58 92 26 40  5 37 93
 64 94 32 67 71 30 46  3  8 95 66 99 98 76 74 24 96 49 83 51 14  4  7 52
 17 39 13 48 61 23 65 91 87 79  2 22 57 19 20 75 25 34  6 56 77 12 28 84
 69  0  1 70]
USER: 3
bought: 182
[62 15 50 97  9 86 85 78 42 55 81 41 59 33 68 27 82 21 47 11 80 10 18 43
 45 29 38 35 90 89 63 73 88 60 16 54 36 44 31 72 53 58 92 26 40  5 37 93
 64 94 32 67 71 30 46  3  8 95 66 99 98 76 74 24 96 49 83 51 14  4  7 52
 17 39 13 48 61 23 65 91 87 79  2 22 57 19 20 75 25 34  6 56 77 12 28 84
 69  0  1 70]
USER: 4
bought: 38
[62

[62 15 50 97  9 86 85 78 42 55 81 41 59 33 68 27 82 21 47 11 80 10 18 43
 45 29 38 35 90 89 63 73 88 60 16 54 36 44 31 72 53 58 92 26 40  5 37 93
 64 94 32 67 71 30 46  3  8 95 66 99 98 76 74 24 96 49 83 51 14  4  7 52
 17 39 13 48 61 23 65 91 87 79  2 22 57 19 20 75 25 34  6 56 77 12 28 84
 69  0  1 70]
USER: 29
bought: 97
[62 15 50 97  9 86 85 78 42 55 81 41 59 33 68 27 82 21 47 11 80 10 18 43
 45 29 38 35 90 89 63 73 88 60 16 54 36 44 31 72 53 58 92 26 40  5 37 93
 64 94 32 67 71 30 46  3  8 95 66 99 98 76 74 24 96 49 83 51 14  4  7 52
 17 39 13 48 61 23 65 91 87 79  2 22 57 19 20 75 25 34  6 56 77 12 28 84
 69  0  1 70]
USER: 30
bought: 35
[62 50  9 97 15 86 85 42 29 41 18 27 55 38 59 81 68 78 21 47 82 35 11 74
 45 33 88 43 89 95 90 80 73 10 92 16 60 54 63 93 72 53 31 26 94 44 37 58
 36 98 30 71 40  3 32  8 64 67  5 46 99 24 17 66 76  7  4 83 52 51 49 61
 39 48 13 96 57 79 14 91 23 87 65 22 19  2 25 75 34 20  6 56 12 77 28 84
  1  0 69 70]
USER: 31
bought: 138
[62 15 50 97  9 86 85

[62  9 50 97 15 85 86 42 38 29 18 41 27 45 55 74 81 59 68 35 82 21 11 47
 88 90 78 33 89 95 80 73 10 92 60 16 93 43 31 53 72 54 63 37 26 98 58 36
 44  3  8 64 32 71 40  5 67 17 99 46 24 94 76 66 30 52  7  4 51 83 49 48
 39 57 13 79 91 14 96 23 65 87 22 61 19  2 25 75 34  6 20 56 12 77 28 84
  1  0 69 70]
USER: 56
bought: 241
[62  9 15 97 50 85 18 42 38 86 41 29 45 27 74 59 35 81 21 68 11 55 33 90
 82 47 88 95 16 93 54 10 92 89 43 73 80 60 31 72 53 26 98 44 78 63 94 36
  8 64 58 17 67 32 30 37 46 40 71  5 99 24  3 76  7  4 51 83 52 66 49 39
 61 13 48 57 91 65 79 87 23 22 14 19  2 75 25 96 34  6 20 56 77 12 28 84
  1  0 69 70]
USER: 57
bought: 88
[62 15 50 97  9 86 85 78 42 55 81 41 59 33 68 27 82 21 47 11 80 10 18 43
 45 29 38 35 90 89 63 73 88 60 16 54 36 44 31 72 53 58 92 26 40  5 37 93
 64 94 32 67 71 30 46  3  8 95 66 99 98 76 74 24 96 49 83 51 14  4  7 52
 17 39 13 48 61 23 65 91 87 79  2 22 57 19 20 75 25 34  6 56 77 12 28 84
 69  0  1 70]
USER: 58
bought: 325
[62 15 50 97  9 86 8

[62 50 97 15  9 86 78 85 42 55 41 59 33 81 68 27 47 82 21 43 29 80 11 10
 18 73 74 45 63 38 35 54 89 88 60 16 30 90 58 92 36 53 72 40 71 26  5 31
 44 32 46 98 95 37 67 14 99 64  8  3 93 66 61 24 51 96 83 52 49  4  7 94
 39 76 13 79 91 23 65 87  2 17 48 22 57 19 20 25 75 34  6 56 12 77 84 28
 69  1  0 70]
USER: 83
bought: 4
[62  9 50 15 97 86 85 38 29 42 41 18 68 55 59 27 74 33 81 82 35 78 47 21
 88 45 89 11 90 37 80 30 73 43 95 10 31 60 54 63 72 53 92 16 26 98 93 36
 58 71  8 64 40 46 32 44  3 66  5 67 99 52 51 24 17 76 83  7  4 49 48 39
 65 14 96 13 57 91 23 87 79 94 22 61  2 19 75 25 34 20  6 56 12 77 28 84
  1 69  0 70]
USER: 84
bought: 85
[62 15 50 97  9 86 85 78 42 55 81 41 59 33 68 27 82 21 47 11 80 10 18 43
 45 29 38 35 90 89 63 73 88 60 16 54 36 44 31 72 53 58 92 26 40  5 37 93
 64 94 32 67 71 30 46  3  8 95 66 99 98 76 74 24 96 49 83 51 14  4  7 52
 17 39 13 48 61 23 65 91 87 79  2 22 57 19 20 75 25 34  6 56 77 12 28 84
 69  0  1 70]
USER: 85
bought: 246
[62 15 50 97  9 86 85 

[62 15 50 97  9 86 85 78 42 55 81 41 59 33 68 27 82 21 47 11 80 10 18 43
 45 29 38 35 90 89 63 73 88 60 16 54 36 44 31 72 53 58 92 26 40  5 37 93
 64 94 32 67 71 30 46  3  8 95 66 99 98 76 74 24 96 49 83 51 14  4  7 52
 17 39 13 48 61 23 65 91 87 79  2 22 57 19 20 75 25 34  6 56 77 12 28 84
 69  0  1 70]
USER: 109
bought: 202
[62 15 50 97  9 86 85 78 42 55 81 41 59 33 68 27 82 21 47 11 80 10 18 43
 45 29 38 35 90 89 63 73 88 60 16 54 36 44 31 72 53 58 92 26 40  5 37 93
 64 94 32 67 71 30 46  3  8 95 66 99 98 76 74 24 96 49 83 51 14  4  7 52
 17 39 13 48 61 23 65 91 87 79  2 22 57 19 20 75 25 34  6 56 77 12 28 84
 69  0  1 70]
USER: 110
bought: 214
[62 15  9 97 45 50 85 42 18 86 29 27 74 55 41 38 81 59 35 68 21 90 11 82
 47 93 33 95 88 16 89 80 98 73 92 60 10 78 43 54 26 53 72 31 63 32  5  8
 17 64 36 58 37 44 24 67 46 40 99 30  3 71 76  7  4 83 51 66 52 94 39 49
 13 61 14 48 79 57 91 22 23 87 65 19 75  2 25 96  6 34 20 56 77 12 28 84
  1  0 69 70]
USER: 111
bought: 137
[62 50  9 97 85 

...

[62 15 50 97  9 86 85 78 42 55 81 41 59 33 68 27 82 21 47 11 80 10 18 43
 45 29 38 35 90 89 63 73 88 60 16 54 36 44 31 72 53 58 92 26 40  5 37 93
 64 94 32 67 71 30 46  3  8 95 66 99 98 76 74 24 96 49 83 51 14  4  7 52
 17 39 13 48 61 23 65 91 87 79  2 22 57 19 20 75 25 34  6 56 77 12 28 84
 69  0  1 70]
USER: 2358
bought: 54
[62  9 86 50 97 94 85 42 78 18 59 55 27 41 44 15 29 68 81 82 33 16 21 47
 35 92 11 88 89 43 80 10 73 63 45 93 95 54 53 72 31 60  3 26 90 32 36 58
 40 17 38 64 67 71  5 30 76  8 96 74 24 99  7  4 46 49 37 83 66 51 61 52
 79 57 39 13 48 23 87 91 14 22 65 98 19  2 20 25 75 34  6 56 12 77 28 84
  0  1 69 70]
USER: 2359
bought: 151
[62 15 50 97  9 86 85 78 42 55 81 41 59 33 68 27 82 21 47 11 80 10 18 43
 45 29 38 35 90 89 63 73 88 60 16 54 36 44 31 72 53 58 92 26 40  5 37 93
 64 94 32 67 71 30 46  3  8 95 66 99 98 76 74 24 96 49 83 51 14  4  7 52
 17 39 13 48 61 23 65 91 87 79  2 22 57 19 20 75 25 34  6 56 77 12 28 84
 69  0  1 70]
USER: 2360
bought: 406
[62 15 50 97  

[62 15 50 97  9 86 85 78 42 55 81 41 59 33 68 27 82 21 47 11 80 10 18 43
 45 29 38 35 90 89 63 73 88 60 16 54 36 44 31 72 53 58 92 26 40  5 37 93
 64 94 32 67 71 30 46  3  8 95 66 99 98 76 74 24 96 49 83 51 14  4  7 52
 17 39 13 48 61 23 65 91 87 79  2 22 57 19 20 75 25 34  6 56 77 12 28 84
 69  0  1 70]
USER: 2384
bought: 61
[62 15 50 97  9 86 85 78 42 55 81 41 59 33 68 27 82 21 47 11 80 10 18 43
 45 29 38 35 90 89 63 73 88 60 16 54 36 44 31 72 53 58 92 26 40  5 37 93
 64 94 32 67 71 30 46  3  8 95 66 99 98 76 74 24 96 49 83 51 14  4  7 52
 17 39 13 48 61 23 65 91 87 79  2 22 57 19 20 75 25 34  6 56 77 12 28 84
 69  0  1 70]
USER: 2385
bought: 265
[62 15 50 97  9 86 85 78 42 55 81 41 59 33 68 27 82 21 47 11 80 10 18 43
 45 29 38 35 90 89 63 73 88 60 16 54 36 44 31 72 53 58 92 26 40  5 37 93
 64 94 32 67 71 30 46  3  8 95 66 99 98 76 74 24 96 49 83 51 14  4  7 52
 17 39 13 48 61 23 65 91 87 79  2 22 57 19 20 75 25 34  6 56 77 12 28 84
 69  0  1 70]
USER: 2386
bought: 44
[62 15 50 97  9

[62 15 50 97  9 86 85 78 42 55 81 41 59 33 68 27 82 21 47 11 80 10 18 43
 45 29 38 35 90 89 63 73 88 60 16 54 36 44 31 72 53 58 92 26 40  5 37 93
 64 94 32 67 71 30 46  3  8 95 66 99 98 76 74 24 96 49 83 51 14  4  7 52
 17 39 13 48 61 23 65 91 87 79  2 22 57 19 20 75 25 34  6 56 77 12 28 84
 69  0  1 70]
USER: 2413
bought: 19
[62 15 50 97  9 86 85 78 42 55 81 41 59 33 68 27 82 21 47 11 80 10 18 43
 45 29 38 35 90 89 63 73 88 60 16 54 36 44 31 72 53 58 92 26 40  5 37 93
 64 94 32 67 71 30 46  3  8 95 66 99 98 76 74 24 96 49 83 51 14  4  7 52
 17 39 13 48 61 23 65 91 87 79  2 22 57 19 20 75 25 34  6 56 77 12 28 84
 69  0  1 70]
USER: 2414
bought: 29
[62 15 50 97  9 86 85 78 42 55 81 41 59 33 68 27 82 21 47 11 80 10 18 43
 45 29 38 35 90 89 63 73 88 60 16 54 36 44 31 72 53 58 92 26 40  5 37 93
 64 94 32 67 71 30 46  3  8 95 66 99 98 76 74 24 96 49 83 51 14  4  7 52
 17 39 13 48 61 23 65 91 87 79  2 22 57 19 20 75 25 34  6 56 77 12 28 84
 69  0  1 70]
USER: 2415
bought: 24
[62 15 50 97  9 

[62 15 50 97  9 86 85 78 42 55 81 41 59 33 68 27 82 21 47 11 80 10 18 43
 45 29 38 35 90 89 63 73 88 60 16 54 36 44 31 72 53 58 92 26 40  5 37 93
 64 94 32 67 71 30 46  3  8 95 66 99 98 76 74 24 96 49 83 51 14  4  7 52
 17 39 13 48 61 23 65 91 87 79  2 22 57 19 20 75 25 34  6 56 77 12 28 84
 69  0  1 70]
USER: 2439
bought: 28
[62 15 50 97  9 86 85 78 42 55 81 41 59 33 68 27 82 21 47 11 80 10 18 43
 45 29 38 35 90 89 63 73 88 60 16 54 36 44 31 72 53 58 92 26 40  5 37 93
 64 94 32 67 71 30 46  3  8 95 66 99 98 76 74 24 96 49 83 51 14  4  7 52
 17 39 13 48 61 23 65 91 87 79  2 22 57 19 20 75 25 34  6 56 77 12 28 84
 69  0  1 70]
USER: 2440
bought: 105
[62 15 50 97 86  9 85 55 42 78 41 81 59 33 68 27 29 82 38 47 45 43 21 80
 10 18 11 35 74 98 90 73 63 60 88 89 54 16 53 72 36 30 58 26  5 37 94 92
 31 40 95 32 71 93 46 67 44  8 14 64 24 66 99 83 51 49  3 61 76 52  4  7
 39 96 13 17 23 79 87 91 65 48  2 22 57 19 20 25 75 34  6 56 12 77 84 28
 69  1  0 70]
USER: 2441
bought: 86
[62 15 50 97  9

Exception: Number of user feature rows does not equal the number of users

In [195]:
user_features.query('user_id == 2455')

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
788,45-54,B,25-34K,Homeowner,2 Adults No Kids,2,None/Unknown,2455


In [196]:
# from lightfm.py
user_ids = np.repeat(np.int32(0), len(test_item_ids))
user_ids.max()+1, user_features.shape[0]

(1, 801)

In [197]:
[i for i in users_purchases.user_id if i not in user_feat_lightfm.index]

[]

In [198]:
from src.metrics import precision_at_k, recall_at_k

N = 10
precisions = []
recalls = []

for user_id in users_purchases.user_id:
    bought = users_purchases.query('user_id == {}'.format(user_id)).purchases.values.tolist()[0]
    try:
        predictions = model.predict(user_ids=user_id, item_ids=test_item_ids,
                                user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                num_threads=4)
    except Exception as e:
        print('USER_ID: {}'.format(user_id))
        print(str(e))
    ranged_items_ids = [int(id_to_itemid[i]) for i in predictions.argsort(axis=-1)[:N]]

    p = precision_at_k(ranged_items_ids, bought)
    r = recall_at_k(ranged_items_ids, bought)
    if p*r != 0:
        print('USER_ID: {}\npredictions:'.format(user_id))
        print_bold_intersect(bought, ranged_items_ids)
        print('precision@k50:', p)
        print('recall@50:', r)

USER_ID: 18
predictions:
822675 825556 821974 819845 819518 824915 824642 821730 821695 822178 
precision@k50: 0.2
recall@50: 0.00625
USER_ID: 19
predictions:
822675 819518 824915 825556 821974 824642 825494 820082 821730 819845 
precision@k50: 0.2
recall@50: 0.001644736842105263
USER_ID: 25
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.005847953216374269
USER_ID: 40
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.0027624309392265192
USER_ID: 52
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.004608294930875576
USER_ID: 56
predictions:
822675 819518 819845 825556 821974 824642 820082 821730 821465 824915 
precision@k50: 0.4
recall@50: 0.008298755186721992
USER_ID: 64
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.00

USER_ID: 438
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.006329113924050633
USER_ID: 442
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.006134969325153374
USER_ID: 443
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.007751937984496124
USER_ID: 451
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.010309278350515464
USER_ID: 464
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.003745318352059925
USER_ID: 469
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.003194888178913738
USER_ID: 473
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 

USER_ID: 957
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.005208333333333333
USER_ID: 963
predictions:
822675 819518 821974 824915 819845 825556 824642 821465 821730 820082 
precision@k50: 0.2
recall@50: 0.00847457627118644
USER_ID: 973
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.004629629629629629
USER_ID: 1005
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.006993006993006993
USER_ID: 1014
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.003355704697986577
USER_ID: 1015
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.005376344086021506
USER_ID: 1023
predictions:
822675 821974 825556 819845 819518 824915 824037 824642 821730 822178 
precision@k5

USER_ID: 1379
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.002074688796680498
USER_ID: 1382
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.008849557522123894
USER_ID: 1385
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.003952569169960474
USER_ID: 1387
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.015625
USER_ID: 1403
predictions:
822675 819518 824915 825556 821974 824642 821730 820082 819845 824037 
precision@k50: 0.2
recall@50: 0.011363636363636364
USER_ID: 1409
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.00641025641025641
USER_ID: 1418
predictions:
822675 819518 825556 821974 824642 824915 821730 820082 819845 821465 
precision@k50: 0.2
re

USER_ID: 1917
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.003215434083601286
USER_ID: 1918
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.006666666666666667
USER_ID: 1930
predictions:
822675 821974 825556 819518 824915 824037 824642 821730 821695 821174 
precision@k50: 0.2
recall@50: 0.007142857142857143
USER_ID: 1934
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.016129032258064516
USER_ID: 1955
predictions:
822675 819518 819845 825556 821974 821788 821730 820082 824642 822178 
precision@k50: 0.2
recall@50: 0.0034129692832764505
USER_ID: 1964
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.0053475935828877
USER_ID: 1980
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision

USER_ID: 2421
predictions:
822675 819518 824915 825556 824642 820082 821974 819845 821730 825494 
precision@k50: 0.2
recall@50: 0.002352941176470588
USER_ID: 2424
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.004048582995951417
USER_ID: 2425
predictions:
822675 819845 821974 825556 819518 824915 824642 824037 821730 822178 
precision@k50: 0.2
recall@50: 0.00625
USER_ID: 2437
predictions:
822675 819845 819518 821465 824915 825556 824642 821974 820082 821007 
precision@k50: 0.2
recall@50: 0.007194244604316547
USER_ID: 2448
predictions:
822675 821974 819518 825556 824915 824642 819845 821730 820082 821695 
precision@k50: 0.2
recall@50: 0.003048780487804878
USER_ID: 2451
predictions:
822675 819845 821974 825556 824915 819518 822178 824642 821730 824037 
precision@k50: 0.2
recall@50: 0.003278688524590164
USER_ID: 2455
Number of user feature rows does not equal the number of users
USER_ID: 2455
predictions:
822675 819845 8

## *Train Precision*

In [125]:
%%time
from lightfm.evaluation import precision_at_k

train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

train_precision 

CPU times: user 11.3 s, sys: 48.2 ms, total: 11.3 s
Wall time: 11.5 s


0.12668024

## *Test Precision*

In [126]:
csr_matrix(test_user_item_matrix).tocsr().shape, sparse_user_item[:1846, :].shape

((1847, 5000), (1846, 5000))

In [157]:
def print_bold_intersect(actual, preds):
    for pred in preds:
        if pred in actual:
            print('\033[1m'+str(pred), end=' ')
            continue
        print('\033[0m'+str(pred), end = ' ')
    print('\033[0m')

In [127]:
test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

test_precision 

0.0034650788

In [161]:
from lightfm.evaluation import recall_at_k, reciprocal_rank, auc_score

In [162]:
test_recall = recall_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

test_recall

0.0012809259397784205

In [163]:
test_rr = reciprocal_rank(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                ).mean()

test_rr

0.016455995

In [164]:
test_auc = auc_score(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 ).mean()

test_auc

0.4973105

очень плохие результаты